In [1]:
# Import libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, iqr
from datetime import datetime
from statsmodels.robust.scale import mad as mad_c
import copy


import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


# Set path for generated figures
fig_path = './figures_4.2/'

# set the scaling factor to 1
def mad(a):
    return mad_c(a, c=1)

In [2]:
# Read data
mydateparser = lambda x: datetime.strptime(x, "%Y-%m-%d")

AAPL = pd.read_csv("AAPL.csv", index_col=0, parse_dates=True, date_parser=mydateparser)
IBM = pd.read_csv("IBM.csv", index_col=0, parse_dates=True, date_parser=mydateparser)
JPM = pd.read_csv("JPM.csv", index_col=0, parse_dates=True, date_parser=mydateparser)
DJI = pd.read_csv("DJI.csv", index_col=0, parse_dates=True, date_parser=mydateparser)

stocks_data = {'APPL': AAPL, 'IBM': IBM, 'JPM': JPM, 'DJI': DJI}

## 4.2 Robust Estimators

This section involves the implementation, analysis and assessment of the following estimators:

- Robust location estimator: median 
- Robust scale estimator: IQR (Interquartile range) and MAD (Median Absolute Deviation)

<a id='4.2.1'></a>
### 4.2.1 Python implementation

We create Python functions for each of the estimator types:

In [3]:
# The way python does the calculation:
# yields in different results

# def compute_median(a):
#     return np.percentile(a, 50)
    
# def compute_iqr(a):
#     return np.percentile(a, 75) - np.percentile(a, 25)

In [4]:
def compute_median(a):
    sorted_a = a.sort_values()
    idx = len(sorted_a) // 2
    if len(a)%2 == 1:
        median = sorted_a.iloc[idx]
    else:
        median = (sorted_a.iloc[idx-1] + sorted_a.iloc[idx]) / 2
    return float(median)


def compute_iqr(a):
    sorted_a = a.sort_values()
    
    idx = len(sorted_a) // 2
    if len(a)%2 == 1:
        q1 = compute_median(sorted_a.iloc[:idx])
        q3 = compute_median(sorted_a.iloc[idx+1:])
    else:
        q1 = compute_median(sorted_a.iloc[:idx])
        q3 = compute_median(sorted_a.iloc[idx:])
    
    return q3 - q1


def compute_mad(a):
    median = compute_median(a)
    deviations = abs(a - median)
    return compute_median(deviations)

Detail on the implementation of each function is given below:

- Median: 
The given series is first sorted in ascending order. Then, if the number of elements is odd, the middle element is returned; else the average of the middle two elements is returned.

- IQR:
The given series is first sorted in ascending order. Then the sorted series is split into two arrays from the middle. The median of both arrays are obtained, which represent the 25th and 75th percentiles, and their difference is returned. Note that the `qr()` function from `scipy.stats` uses `np.percentile()` function to calculate the values of 25th and 75th percentiles, which results in slightly different values due to a difference in the calculation method.

- MAD:
The median of the given series is computed using the {\tt compute\_median()} function. Then, a new series is obtained by calculating the absolute difference between the median and each element of the series. The median of the new series is returned.


<a id='4.2.2'></a>
### 4.2.2 Computational efficiency

Computational efficiencies of the different estimators are discussed below:

- Median: Sorting the series takes $\mathcal{O}(n\log(n))$ operations, and then finding the middle value takes $\mathcal{O}(1)$ operations. The overall complexity of this estimator is then $\mathcal{O}(n\log(n))$.

- IQR: Sorting the series takes $\mathcal{O}(n\log(n))$ operations. After splitting the series, finding the medians of the resulting two series each takes $\mathcal{O}(n\log(n))$ (it is actually less than that since the given series are both already sorted) operations. The rest of the operations are of $\mathcal{O}(1)$. The overall complexity of this estimator is then $\mathcal{O}(n\log(n))$.

- MAD: Computing the median of the series takes $\mathcal{O}(n\log(n))$ operations. Then obtaining the new series with the absolute deviations from the median takes $\mathcal{O}(n))$ operations. Computing the median of the resulting series takes another $\mathcal{O}(n\log(n))$ operations. The overall complexity of this estimator is then also $\mathcal{O}(n\log(n))$.



Note that the computational complexities of computing the mean and the standard deviation are of $\mathcal{O}(n)$, which is less than the complexities of the estimators discussed in this section.

<a id='4.2.3'></a>
### 4.2.3 Breakdown Points

A robust estimator should not be affected too much by somewhat larger deviations from the model. One of the properties that defines the robustness of an estimator is its *breakdown point*, which is defined as the maximum allowed fraction of outliers that an estimator can tolerate and has a value between 0 and 0.5. In other words, the breakdown point is the fraction of the data that can be contaminated without making the estimator's estimation "very bad". The breakdown points of different estimators are given in the table below.

In [5]:
df = {'mean': 0., 'std. dev.': 0., 'median': 0.5, 'MAD': 0.5, 'IQR':0.25}
pd.DataFrame(df, index=["Breakdown points"])

,mean,std. dev.,median,MAD,IQR
Breakdown points,0.0,0.0,0.5,0.5,0.25


The median has a breakdown point of 0.5, so it is very robust against outliers. Since the median is the middle value in the data set, even if the half of the dataset was contaminated with extremely large (or low) values, it would still estimate a valid point within the dataset. The sample mean as an estimator of the average, however, is largely susceptible to outliers, and even a single extreme data value would cause an irrelevant estimation.

The median absolute deviation also has a breakdown point of 0.5 from a similar argument for the median. If the half of the dataset had extremely large values, the median would still have a viable value, half of the deviations from the median would have viable values, and the median of the deviations would again have a viable value in representing the dataset. Standard deviation as an estimator of the deviation, on the other hand, would break down in the presence of a single extreme value.

Finally, the interquartile range has a breakdown point of 0.25. Because it estimates the data points at 25th and 75th percentiles, until a quarter of the data points are contaminated with extremely large (or low) values, it gives a robust estimation; however, after that point, it will break down.